# Stifness matrix of the P2 triangle element.

In [1]:
x,y=var("x,y")

The barycentric coordinates on the reference triangle:

In [2]:
lamb=[1-x-y, x,y]

The P2 basis functions:

In [3]:
phi=[2*l*(l-1/2) for l in lamb]+[4*lamb[i]*lamb[(i+1)%3] for i in range(0,3)]

Verify that basis functions value is zero or one where it should be:

In [4]:
points=[[0,0],[1,0],[0,1],[1/2,0],[1/2,1/2],[0,1/2]]
A=matrix(6,6)
for i in range(0,6):
    f=phi[i]
    for j in range(0,6):
        A[i,j]=f(x=points[j][0],y=points[j][1])
assert(A==identity_matrix(6,6))

Gradients of  basis functions:

In [5]:
grads=[[diff(f,x),diff(f,y)] for f in phi]

Gradients at quadrature points (midpoints of the edges):

$gq_{i,j}$= gradient of ith function at jth quadrature point.

In [6]:
quad=[[1/2,0],[1/2,1/2],[0,1/2]]
gq=[[(f[0](x=q[0],y=q[1]),f[1](x=q[0],y=q[1])) for q in quad]  for f in grads]  

Transformation matrix J from reference to current triangle:

In [7]:
x0,x1,x2,y0,y1,y2=var("x0,x1,x2,y0,y1,y2")# current triangle
J=matrix([[x1-x0,x2-x0],[y1-y0,y2-y0]])
det=J.determinant().simplify_rational()
Jinv=J.inverse().simplify_rational()
JinvDet=Jinv*det #factorize determinant.
# transpose of J^-1 multiplied by determinant:
JinvDetTrans=JinvDet.transpose()

Verify that 'everything' is ok on the reference triangle:

In [8]:
assert(det(x0=0,x1=1,y0=0,y1=0,x2=0,y2=1)==1)
assert(JinvDetTrans(x0=0,x1=1,y0=0,y1=0,x2=0,y2=1)==identity_matrix(2,2))

Exact integration of a function f(x,y) on the reference triangle:

In [9]:
integreftr=lambda f: f.integral(x,0,1-y).integral(y,0,1)

Stiffness matrix on reference triangle, exactly computed:

In [10]:
M=matrix(QQ,6,6)
for i in range(0,6):
    for j in range(0,6):
        gigj= grads[i][0]*grads[j][0]+grads[i][1]*grads[j][1]
        M[i,j]=integreftr(gigj)        

Stiffness matrix, numerically computed:

In [11]:
MN=matrix(SR,6,6)
Ji=JinvDetTrans
for i in range(0,6):
    for j in range(0,6):
        s=0
        for k in range(0,3):
            V=Ji*vector(gq[i][k])
            W=Ji*vector(gq[j][k])
            s+=V.dot_product(W)
        MN[i,j]=s
MN*=(1/(6*det))

Verify that stiffness matrix as computed by quadrature is equal to the  'exact' matrix (on the reference element !). Recall that our numerical quadrature is exact for 2nd order polynomial and that, here, we compute in $\mathbb{Q}$ (all computations are exact).

In [12]:
assert(MN(x0=0,x1=1,y0=0,y1=0,x2=0,y2=1)==M)

Verify also that if v=[1,1,1,1,1,1], M.V==0 and MN.V=0.

In [13]:
x=vector([1,1,1,1,1,1])
assert((M*x).is_zero())
assert((MN*x).simplify_rational().is_zero())

How is the stiffness matrix on the reference triangle?

In [14]:
print(M)

[   1  1/6  1/6 -2/3    0 -2/3]
[ 1/6  1/2    0 -2/3    0    0]
[ 1/6    0  1/2    0    0 -2/3]
[-2/3 -2/3    0  8/3 -4/3    0]
[   0    0    0 -4/3  8/3 -4/3]
[-2/3    0 -2/3    0 -4/3  8/3]


Define a function that will return the stiffness matrix, for x=[,,], y=[,,] containing the coordinates  of the summits of a triangle (usefull for debugging codes!)

In [15]:
def Stiff(x,y):
    return MN(x0=x[0],x1=x[1],y0=y[0],y1=y[1],x2=x[2],y2=y[2])

Do we again compute the same matrix?

In [16]:
x=vector([0,1,0])
y=vector([0,0,1])
assert(Stiff(x,y)==M)

Now, the matrix should be invariant when dilating a triangle; let's
verify this with the reference triangle:

In [17]:
for i in range(0,200):
    dilat= abs(QQ.random_element(100,100)) # a "random" number.
    if dilat != 0:
        x1=dilat*x
        y1=dilat*y
        assert(Stiff(x1,y1) == M)

Verify that the Stiffness matrix is invariant under some rotations.
We must rotate by angles whose sinus and cosinus are Algebraic Numbers,
 and compute in (real) Algebraic Numbers (AA for SageMath), so as to make
 again exact computations:

In [18]:
tetas=[pi/12,pi/6,pi/4,pi/3,pi/2,2*pi/3]
for teta in tetas:
    Rot=matrix(AA,[[cos(teta),-sin(teta)],[sin(teta),cos(teta)]])
    V=[Rot*vector(AA,[x[i],y[i]]) for i in range(0,3)]
    assert(Stiff([s[0] for s in V],[s[1] for s in V])==M)

--End--